In [14]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [15]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

In [ ]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]
df_home 

In [17]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Alemania,2.750000,0.886364
Argentina,0.555556,4.111111
Australia,1.461538,1.923077
Brasil,1.941176,1.176471
Camerún,1.500000,1.500000
Canadá,1.259259,1.925926
Chile,0.666667,1.666667
China,1.606061,0.969697
China Taipéi,0.500000,3.750000


Función predicción

In [18]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [19]:
print(predict_points('Inglaterra', 'Estados Unidos'))
print(predict_points('Argentina', 'Marruecos')) # Marruecos vs Team X -> 0 points to both
print(predict_points('Colombia', 'España')) 

(0.4791290285407772, 2.3861628168887195)
(0, 0)
(0.9712746801019329, 1.7071459668421043)


PREDICCIÓN ⚽

FASE DE GRUPOS

In [20]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [ ]:
df_fixture_group_48

In [23]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts.'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts.'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts.', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts.']]
    dict_table[group] = dict_table[group].round(0)

In [24]:
dict_table['H']

,Team,Pts.
0,Alemania,5.0
1,Colombia,2.0
2,Corea del Sur,1.0
3,Marruecos,0.0


In [44]:
df_fixture_quarter.replace({'OF':'Winners'})
#for i in df_fixture_quarter:
 # print(i)
  #partido = str(i+48)
  #df_fixture_quarter.replace({f'OF{i}':f'Winners {partido}'})

,home,score,away,year,winner
56,OF1,Partido 57,OF3,2023,?
57,OF2,Partido 58,OF4,2023,?
58,OF5,Partido 59,OF7,2023,?
59,OF6,Partido 60,OF8,2023,?


In [11]:
df_fixture_knockout

,home,score,away,year
48,1A,Partido 49,Plantilla:Geodatos 2C ?,2023
49,1C,Partido 50,2A,2023
50,1E,Partido 51,2G,2023
51,1G,Partido 52,2E,2023
52,1D,Partido 54,2B,2023
53,1B,Partido 53,2D,2023
54,1H,Partido 56,2F,2023
55,1F,Partido 55,2H,2023


In [ ]:
dict_table

In [28]:
df_fixture_knockout.replace({'Plantilla:Geodatos 2C ?':'2C'}, inplace=True)

In [29]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'1{group}':group_winner,
                                 f'2{group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Suiza,Partido 49,España,2023,?
49,Costa Rica,Partido 50,Noruega,2023,?
50,Estados Unidos,Partido 51,Suecia,2023,?
51,Italia,Partido 52,Países Bajos,2023,?
52,China,Partido 54,Canadá,2023,?
53,Australia,Partido 53,Inglaterra,2023,?
54,Alemania,Partido 56,Brasil,2023,?
55,Francia,Partido 55,Colombia,2023,?


Obtener ganador

In [30]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [31]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Suiza,Partido 49,España,2023,Suiza
49,Costa Rica,Partido 50,Noruega,2023,Noruega
50,Estados Unidos,Partido 51,Suecia,2023,Estados Unidos
51,Italia,Partido 52,Países Bajos,2023,Italia
52,China,Partido 54,Canadá,2023,China
53,Australia,Partido 53,Inglaterra,2023,Inglaterra
54,Alemania,Partido 56,Brasil,2023,Alemania
55,Francia,Partido 55,Colombia,2023,Francia


Cuartos de final

In [34]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [35]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,OF1,Partido 57,OF3,2023,?
57,OF2,Partido 58,OF4,2023,?
58,OF5,Partido 59,OF7,2023,?
59,OF6,Partido 60,OF8,2023,?
